In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import folium
from folium.plugins import MarkerCluster

import json
from pandas.io.json import json_normalize

%matplotlib inline

In [9]:
data = json_normalize(json.load(open('history.json', 'r'))['locations'])
#data = pd.read_csv('history500.csv')
#f = open('history500.csv', 'r')
#data = pd.read_csv(f)
#df = pd.read_json('history.json')
#df.head(10)

,locations
0,"{'timestampMs': '1329336160513', 'latitudeE7':..."
1,"{'timestampMs': '1329336171017', 'latitudeE7':..."
2,"{'timestampMs': '1329339830733', 'latitudeE7':..."
3,"{'timestampMs': '1329343459578', 'latitudeE7':..."
4,"{'timestampMs': '1329374288615', 'latitudeE7':..."
5,"{'timestampMs': '1329374348733', 'latitudeE7':..."
6,"{'timestampMs': '1329374469174', 'latitudeE7':..."
7,"{'timestampMs': '1329374647625', 'latitudeE7':..."
8,"{'timestampMs': '1329374827647', 'latitudeE7':..."
9,"{'timestampMs': '1329375132758', 'latitudeE7':..."


In [10]:
# df.iloc[:2500].to_json('hi2500.json')

In [3]:
d_min, d_max = data.timestampMs.min(), data.timestampMs.max()
print('Первая отметка:', str(pd.to_datetime(int(d_min) * 1000000).date()))
print('Пеоследняя отметка:', str(pd.to_datetime(int(d_max) * 1000000).date()))

Первая отметка: 2012-02-15
Пеоследняя отметка: 2012-02-26


In [12]:
data['longitude'] = data.longitudeE7 / 10000000
data['latitude'] = data.latitudeE7 / 10000000
data.drop(['longitudeE7', 'latitudeE7'], axis=1, inplace=True)

In [13]:
data.drop(['altitude', 'verticalAccuracy'], axis=1, inplace=True)  # Дропнули ненужные столбцы
data.fillna(0, inplace=True)   # Убили все неопределённые значения
data = data[data['velocity'] == 0]  # Убрали значения, где были перемещения в пространстве. Оставили только статику
data = data[data['accuracy'] < 1000]  # Убрали значения, где accuracy больше 1000, т.е. неточные
data = data.set_index(pd.to_datetime(data.timestampMs.map(int) * 1000000)).drop('timestampMs', axis=1)   # Сконвертировали время в нормальный вид
data.index.name = 'time'

data.head()

,Unnamed: 0,accuracy,activity,heading,velocity,longitude,latitude
time,,,,,,,
2012-02-15 20:02:40.513,0,31.0,0.0,0.0,0.0,30.298478,59.813564
2012-02-15 20:02:51.017,1,20.0,0.0,0.0,0.0,30.298513,59.813568
2012-02-15 21:03:50.733,2,905.0,0.0,0.0,0.0,30.304697,59.819625
2012-02-15 22:04:19.578,3,905.0,0.0,0.0,0.0,30.304697,59.819625
2012-02-16 06:38:08.615,4,46.0,0.0,0.0,0.0,30.297762,59.813330


In [14]:
dmt = data.resample('10T').last().dropna(how='all')  
dhr = data.resample('H').last().dropna(how='all')  
dh12 = data.resample('12H').last().dropna(how='all')
ddy = data.resample('D').last().dropna(how='all') 

ddy.shape, dhr.shape, dmt.shape, data.shape   # сделали несколько дополнительных датафреймов с пониженной точностью

((12, 7), (155, 7), (244, 7), (391, 7))

In [15]:
def getmap(data, radius=5, color='red', opacity=0.9, zoom=12, cluster=False):
    
    map = folium.Map(location=[data.latitude.median(), data.longitude.median(), ], zoom_start=zoom)
    lat, lon = data.latitude, data.longitude
    date = data.index    

    if cluster:
        final_map = MarkerCluster().add_to(map)
    else:
        final_map = map 
        

    for lat, lon in zip(lat, lon):
        folium.CircleMarker(location=(lat, lon), 
                            radius=radius, 
                            stroke=False, 
                            fill_color=color, 
                            fill_opacity=opacity).add_to(final_map) 

    return map

# map = getmap(ddy).save('map.html')

In [16]:
# Рисуем карту своих самых частых местонахождений
getmap(ddy, radius=5, color='red', opacity=0.05, zoom=10)

In [17]:
# Вариант с кластерами
getmap(ddy, radius=5, color='red', opacity=0.05, zoom=10, cluster=True)

In [18]:
# Прожорливый вариант. Работает только после сохранения в HTML
# getmap(dhr, radius=3, color='year', opacity=.7).save('Map.html')